In [1]:
import pandas as pd
import numpy as np
import xlrd

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('orig_data/mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('orig_data/mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('orig_data/mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('orig_data/mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('orig_data/mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('orig_data/mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('orig_data/mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('orig_data/mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('orig_data/mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('orig_data/mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [4]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['run_line_close'] = dfnew[0].apply(float)
        df['run_line_odds_close'] = dfnewer.apply(int)
    except KeyError:
        pass

In [5]:
for df in dfList:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.rename({"vh": "visitor_home", "final": "team_run_total", "close":"money_line_close", "open":"money_line_open", "openou":"over_under_line_open", "open_ou":"over_under_line_open", "unnamed:_18":"over_under_odds_open", "unnamed:_19":"over_under_odds_open", "close_ou":"over_under_line_close", "closeou":"over_under_line_close", "unnamed:_20":"over_under_odds_close", "unnamed:_21":"over_under_odds_close"}, axis='columns', inplace=True)    

In [6]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['over_under_line_close'] = df['over_under_line_close'].apply(convt)

In [7]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                totRuns.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                counter += row.team_run_total
            else:
                counter += row.team_run_total
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['total_runs_game'] = totRuns

In [8]:
for df in dfList:
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                RunDif.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                team1 = row.team_run_total
            else:
                team2 = row.team_run_total
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['run_dif_game'] = RunDif

In [10]:
df1 = pd.concat(dfList, sort=True, ignore_index=True)

In [11]:
df1.drop(['year', 'rl', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th'], axis=1, inplace=True)

In [12]:
df1 = df1[df1['visitor_home'] != 'N']

In [13]:
colNew = []
i = 0
for index, row in df1.iterrows():
    if index == 0:
        pass
    elif index % 2 == 0:
        i += 1
    colNew.append(i)
df1['gameno'] = colNew
df1.replace('LOS','LAD',inplace=True)
df1.replace('CUB','CHC',inplace=True)
df1.replace('SDG','SD',inplace=True)
df1.replace('SFO','SF',inplace=True)
df1.replace('KAN','KC',inplace=True)
df1.replace('TAM','TB',inplace=True)

In [14]:
df1 = df1.reindex(columns=['date', 'gameno', 'team', 'visitor_home', 'team_run_total', 'total_runs_game', 'money_line_close', 'money_line_open', 'over_under_line_close', 'over_under_line_open', 'over_under_odds_close', 'over_under_odds_open', 'pitcher', 'rot', 'run_dif_game', 'run_line_close', 'run_line_odds_close'])

In [15]:
df1.replace(['NL', 'nan'], np.nan, inplace=True)

In [45]:
#merge visitor and home rows together
df_v = df1.loc[df1['visitor_home'] == 'V']
df_h = df1.loc[df1['visitor_home'] == 'H']
df_h.columns = ['h_date', 'gameno', 'h_team', 'home', 'h_team_run_total',
                   'h_total_runs_game', 'h_money_line_close', 'h_money_line_open',
                   'under_line_close', 'under_line_open',
                   'under_odds_close', 'under_odds_open', 'h_pitcher', 'h_rot',
                   'h_run_dif_game', 'h_run_line_close', 'h_run_line_odds_close']
df_v.columns = ['date', 'gameno', 'v_team', 'visitor', 'v_team_run_total',
                   'total_runs_game', 'v_money_line_close', 'v_money_line_open',
                   'over_line_close', 'over_line_open',
                   'over_odds_close', 'over_odds_open', 'v_pitcher', 'v_rot',
                   'v_run_dif_game', 'v_run_line_close', 'v_run_line_odds_close']

df3 = pd.merge(df_v, df_h, on='gameno', how='right')

In [46]:
#drop unnecessary columns
df3.drop(['h_total_runs_game','h_date'], axis=1, inplace=True)

In [47]:
#reindex columns
df_final = df3.reindex(columns = ['date', 'gameno', 
             
             'visitor', 'v_team', 'v_team_run_total', 
             'v_money_line_close', 'v_money_line_open', 
             'v_run_line_close', 'v_run_line_odds_close', 
             'v_pitcher', 'v_rot', 'v_run_dif_game',
             
             'home', 'h_team', 'h_team_run_total', 
             'h_money_line_close', 'h_money_line_open', 
             'h_run_line_close', 'h_run_line_odds_close',
             'h_pitcher', 'h_rot', 'h_run_dif_game',
             
             'total_runs_game', 
             'over_line_close', 'over_line_open', 'over_odds_close', 'over_odds_open', 
             'under_line_close', 'under_line_open', 'under_odds_close', 'under_odds_open' 
            ])

In [48]:
df_final

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,...,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
0,404,0,V,NYY,7.0,107.0,104,NaN,NaN,CSABATHIA-L,...,2.0,16.0,9.0,9.0,-105,-116,9.0,9.0,-115,-104
1,405,1,V,PHI,11.0,-200.0,-200,NaN,NaN,RHALLADAY-R,...,-10.0,12.0,7.5,7.5,-125,100,7.5,7.5,105,-120
2,405,2,V,MIA,1.0,-102.0,105,NaN,NaN,JJOHNSON-R,...,6.0,8.0,7.0,7.0,-130,-125,7.0,7.0,110,105
3,405,3,V,STL,11.0,-143.0,-155,NaN,NaN,CARPENTER-R,...,-5.0,17.0,7.5,7.5,-105,-105,7.5,7.5,-115,-115
4,405,4,V,LAD,5.0,-126.0,-155,NaN,NaN,VPADILLA-R,...,6.0,16.0,8.5,8.5,-120,-110,8.5,8.5,100,-110
5,405,5,V,COL,5.0,-105.0,110,NaN,NaN,UJIMENEZ-R,...,-2.0,8.0,7.5,7.5,105,-105,7.5,7.5,-125,-115
6,405,6,V,CHC,5.0,110.0,115,NaN,NaN,CZAMBRANO-R,...,11.0,21.0,8.5,8.5,100,-110,8.5,8.5,-120,-110
7,405,7,V,SD,3.0,166.0,155,NaN,NaN,JGARLAND-R,...,3.0,9.0,8.0,8.0,-120,-110,8.0,8.0,100,-110
8,405,8,V,SF,5.0,-147.0,-130,NaN,NaN,TLINCECUM-R,...,-3.0,7.0,7.0,7.5,-110,105,7.0,7.5,-110,-125
9,405,9,V,CLE,0.0,127.0,150,NaN,NaN,WESTBROOK-R,...,6.0,6.0,9.0,9.0,-105,-105,9.0,9.0,-115,-115


In [ ]:
pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'dataset.pickle'))
df1.to_pickle(pickle_path)